In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
from os import listdir
import re

### Retrieve data files list

In [2]:
folder = 'data/'
# get all file names
files = [f for f in listdir(folder) if f[-4:]=='.png']
# load one to get the dimensions
image = mpimg.imread(f'{folder}{files[0]}')
# get dimensions
shape = list(image.shape)
# remove alpha if necessary
if (shape[2]==4):
    shape[2] = 3
n = len(files)

### Read in data

In [3]:
y = np.zeros(shape=n,dtype=int)
X = np.zeros(shape=(n,*shape))
for i in range(len(files)):
    # get the hour from the file name
    y[i] = re.search('^([0-9]*)_',files[i])[1]
    # get the pixels, remove the alpha if needed and convert to 0-255
    X[i] = (mpimg.imread(f'{folder}{files[i]}')[:,:,:shape[2]] )
    
# transform y to label encoded
temp = y.copy()
y = np.zeros(shape=(n,12),dtype=int)
for i in range(n):
    y[i,temp[i]-1] = 1   

### Split data to training and test sets

In [4]:
# note, the input files are already shuffled
train = 0.8
cutoff = int(len(X) * train)

X_train = X[:cutoff]
y_train = y[:cutoff]

X_test = X[cutoff:]
y_test = y[cutoff:]

### Network

In [5]:
import tensorflow as tf

In [11]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def new_bias(length):
    return tf.Variable(tf.constant(0.0,shape=[length]))
    
def new_conv_layer(X,channels,k_size,n_kernels,use_pooling=True):
    shape=[k_size,k_size,channels,n_kernels]
    kernels = new_weights(shape)
    bias = new_bias(n_kernels)
    
    layer = tf.nn.conv2d(input=X,filter=kernels,strides=[1,1,1,1],padding='SAME')
    layer+= bias
    
    if use_pooling:
        layer = tf.nn.max_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    layer = tf.nn.relu(layer)
    
    return layer, kernels

def flatten(layer):
    shape = layer.shape
    num_features = np.array(shape[1:4],dtype=int).prod()
    layer_flat = tf.reshape(layer,[-1,num_features])
    return layer_flat,num_features

def new_fc_layer(X,num_inputs,num_outputs,use_relu=True,drop_out=None):
    weights = new_weights([num_inputs,num_outputs])
    bias = new_bias(num_outputs)
    layer = tf.matmul(X,weights) + bias
    if (use_relu):
        layer = tf.nn.relu(layer)
    if (drop_out is not None):
        layer = tf.nn.dropout(layer,drop_out)        
    return layer

img_size = shape[0]
channels = shape[2]
img_size_flat = np.array(shape).prod()

x_image = tf.placeholder(tf.float32,shape=[None,img_size,img_size,channels],name='x_image')

y_true = tf.placeholder(tf.float32,shape=[None,12],name='y_true')

nk1 = 64
nk2 = 32

layer_conv1, conv1_weights = new_conv_layer(x_image,channels,3,nk1,False)  
layer_conv2, conv2_weights = new_conv_layer(layer_conv1,nk1,3,nk2,True)  
layer_flat,n_n = flatten(layer_conv2)

n_fc = 256

layer_fc1 = new_fc_layer(layer_flat,n_n,n_fc,True,0.5)

layer_fc2 = new_fc_layer(layer_fc1,n_fc,12,False)

y_pred = tf.nn.softmax(layer_fc2)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

pred_labels = tf.math.round(y_pred)
correct_pred = tf.equal(pred_labels,y_true)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [12]:
session = tf.Session()

In [13]:
session.run(tf.global_variables_initializer())

In [14]:
def optimize(num_iterations,X,y,batch_size):
    batches = int(len(X)/batch_size)
    for i in range(num_iterations):    
        for j in range(batches):
            x_batch = X[j*batch_size:(j+1)*batch_size]
            y_batch = y[j*batch_size:(j+1)*batch_size]
            feed_dict = {x_image: x_batch,y_true:y_batch}
            session.run(optimizer,feed_dict)
        cst,acc = session.run([cost,accuracy],feed_dict)
        print(f'Iteration {i}', f'cost: {cst}, accuracy: {acc}')

In [15]:
optimize(num_iterations=20,X=X_train,y=y_train,batch_size=50)

Iteration 0 cost: 2.4695303440093994, accuracy: 0.9166666865348816
Iteration 1 cost: 1.8936419486999512, accuracy: 0.9183333516120911
Iteration 2 cost: 1.1975780725479126, accuracy: 0.9283333420753479
Iteration 3 cost: 0.9264894127845764, accuracy: 0.9416666626930237
Iteration 4 cost: 0.5848761796951294, accuracy: 0.9583333134651184
Iteration 5 cost: 0.5571196675300598, accuracy: 0.9700000286102295
Iteration 6 cost: 0.32585960626602173, accuracy: 0.9766666889190674
Iteration 7 cost: 0.27483734488487244, accuracy: 0.9883333444595337
Iteration 8 cost: 0.19561710953712463, accuracy: 0.9916666746139526
Iteration 9 cost: 0.06376275420188904, accuracy: 0.996666669845581
Iteration 10 cost: 0.10671758651733398, accuracy: 0.9933333396911621
Iteration 11 cost: 0.04543231800198555, accuracy: 0.996666669845581
Iteration 12 cost: 0.10767029970884323, accuracy: 0.9933333396911621
Iteration 13 cost: 0.07193771004676819, accuracy: 0.9950000047683716
Iteration 14 cost: 0.16845545172691345, accuracy: 0.

In [16]:
pred = session.run(y_pred,{x_image:X_test})

In [17]:
flag_p = np.argmax(pred,axis=1)
flag_r = np.argmax(y_test,axis=1)

In [18]:
(flag_p == flag_r).mean()

0.82

In [19]:
session.close()